In [15]:
import numpy as np
import pandas as pd

### First Practice Activity of a Recommendation System

To begin, my objective is learn about RecSystems with Reinforcement Learning, but as i don't know anything about RecSys or RL, i decided to understanding each term separately and, after that, finally study the two as a unique theme. Therefore, i will start with Recommendation System, due to the fact of be more simple than RL, beginning with all important definitions in these area of study.

#### First Ideas
The basis of the recommendation system is "What do you want to recommend, for whom, and how?" you can find many ways to make recommendations.

For example, if you have a client who watches a horror movie, you have two main ways to choose a recommendation for him. The first is to recommend other horror movies like the first one, i.e., similar items(films) to the original article in the interaction—that is an item-focused approach.

In other cases, you wish to recommend movies watched by other users similar to the client who watches that horror movie; in that case, you use a User-focused approach.

You can apply these approaches to different types of recommendation systems.


Despite that, you have two ways to make a recommendation system: collaborative and content.

##### Collaborative

Collaborative Methods use past interactions between users and items to make new recommendations using a "User-Item Interactions Matrix." In the middle of the recommendation, it uses the proximity between the users/items to predict. Is it possible to use a model-based approach or a metric to calculate the similarity and just use this metric.

It has some issues, like problems with new Users and Items, but with other methods specific to further information that can surpass this problem.

A lot of methods using a collaborative approach, such as:

        - Memory Based
        - Model-Based

##### Content

Content methods use additional information about users and/or items. In the movie example, you can have information about the Release Year, the Director, the Principal Actors, and many other data.

These methods have much more information and don't suffer with new Users or Items too much because they 

### Practice

I will use the most simple recommendation system for this first recommendation system, a memory-based collaborative model. These recommendation systems use just similarity metrics to make a recommendation, i.e., they don't have any ML model in the middle of the process as the simplest method is the most interesting way to new students like me.

One of the problems of that method is the bad scalability with the new users and new movies, because of the pivot table.

In [16]:
names = pd.read_csv("./movie_information/movie.csv")

names.head()

# how i gonna use a collaborative approach, i need just the information about the interaction user-item
# for later visualization i will include the title in the matrix, but just for make the visualization easiest

names = names.drop("genres", axis = 1)

In [17]:
interactions = pd.read_csv("./movie_information/rating.csv")

# that interactions are the most import information for the collaborative approach

interactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  object 
dtypes: float64(1), int64(2), object(1)
memory usage: 610.4+ MB


In [18]:
data = pd.merge(names, interactions)

data.head()

,movieId,title,userId,rating,timestamp
0,1,Toy Story (1995),3,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),6,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),8,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),10,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),11,4.5,2009-01-02 01:13:41


In [19]:
data["release_year"] = data["title"].str[-7:]

data["title"] = data["title"].str[:-7]

data

,movieId,title,userId,rating,timestamp,release_year
0,1,Toy Story,3,4.0,1999-12-11 13:36:47,(1995)
1,1,Toy Story,6,5.0,1997-03-13 17:50:52,(1995)
2,1,Toy Story,8,4.0,1996-06-05 13:37:51,(1995)
3,1,Toy Story,10,4.0,1999-11-25 02:44:47,(1995)
4,1,Toy Story,11,4.5,2009-01-02 01:13:41,(1995)
...,...,...,...,...,...,...
20000258,131254,Kein Bund für's Leben,79570,4.0,2015-03-30 19:32:59,(2007)
20000259,131256,"Feuer, Eis & Dosenbier",79570,4.0,2015-03-30 19:48:08,(2002)
20000260,131258,The Pirates,28906,2.5,2015-03-30 19:56:32,(2014)
20000261,131260,Rentun Ruusu,65409,3.0,2015-03-30 19:57:46,(2001)


In [20]:
del data["timestamp"]

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000263 entries, 0 to 20000262
Data columns (total 5 columns):
 #   Column        Dtype  
---  ------        -----  
 0   movieId       int64  
 1   title         object 
 2   userId        int64  
 3   rating        float64
 4   release_year  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 915.5+ MB


In [21]:
# 20Million taking just 5 Million to reduce the dimensionality
sampled_data = data.iloc[:5000000,:]

In [22]:
sampled_data["title"].value_counts()

Pulp Fiction                 67310
Forrest Gump                 66172
Shawshank Redemption, The    63366
Silence of the Lambs, The    63299
Jurassic Park                59715
                             ...  
Honey Moon (Honigmond)          12
Boy Called Hate, A              11
Roula                           11
Girl in the Cadillac            10
Criminals                        6
Name: title, Length: 886, dtype: int64

In [23]:
sampled_data["userId"].value_counts()

124052    843
83090     786
128653    753
118205    701
46663     667
         ... 
89906       1
134454      1
92207       1
87256       1
111487      1
Name: userId, Length: 137065, dtype: int64

In [24]:
sampled_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000000 entries, 0 to 4999999
Data columns (total 5 columns):
 #   Column        Dtype  
---  ------        -----  
 0   movieId       int64  
 1   title         object 
 2   userId        int64  
 3   rating        float64
 4   release_year  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 228.9+ MB


In [25]:
# Tables used in collaborative approaches have the format: Rows = users, Columns = Items, so we need to create a Pivot table with our data

pivot_table = sampled_data.pivot_table(index = ["userId"], columns = ["title"], values = "rating")

pivot_table

title,'Til There Was You,1-900 (06),"301, 302 (301/302)",8 Seconds,Above the Rim,Ace Ventura: Pet Detective,Ace Ventura: When Nature Calls,Across the Sea of Time,Addams Family Values,"Addiction, The",...,Wings of Courage,With Honors,Wolf,Women Robbers (Diebinnen),"Wonderful, Horrible Life of Leni Riefenstahl, The (Macht der Bilder: Leni Riefenstahl, Die)","Wooden Man's Bride, The (Yan shen)","World of Apu, The (Apur Sansar)",Wyatt Earp,Yankee Zulu,"Young Poisoner's Handbook, The"
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138489,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138491,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
def search_movie_name(search):
   result = (pivot_table.columns
          .to_series()
          .reset_index(drop = True)
          .loc[pivot_table.columns.str.contains(search)]).reset_index(drop = True)
   #print(result)
   return result.loc[0]

def get_recommendation(movie_name):
    if not(movie_name in pivot_table.columns):
        print("O nome não está presente.")
        return
    movie = pivot_table[movie_name]
    similarity = pivot_table.corrwith(movie)
    similarity = similarity.sort_values(ascending=False)
    print(similarity.iloc[1:11])

In [72]:
movie_name = search_movie_name("Ace")
get_recommendation(movie_name)

title
Headless Body in Topless Bar                  0.914991
Bye-Bye                                       0.881422
Ace Ventura: When Nature Calls                0.721531
War Stories                                   0.693167
Dumb & Dumber (Dumb and Dumber)               0.593361
Costa Brava                                   0.578394
Every Other Weekend (Un week-end sur deux)    0.560063
Shadow of Angels (Schatten der Engel)         0.523733
Modern Affair, A                              0.502173
Jar, The (Khomreh)                            0.499000
dtype: float64


This method was a pratical example of a item based collaborative recommendation.

Now i gonna use a user based collaborative approach. Using the same dataset.


#### Possível melhorias, testar outras métricas, usar KNN ou K-Means, não pensei ainda para user based, utilizar estruturas como sparse table para armazenar a pivot
#### Pegar como base o código para desenvolver o user-based
